In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv(r'/content/bank_traffic_data.csv')

In [ ]:
df.head()

,timestamp,count,window
0,2019-09-26 02:00:00+02:00,764,1
1,2019-09-27 02:00:00+02:00,0,1
2,2019-09-28 02:00:00+02:00,0,1
3,2019-09-29 02:00:00+02:00,1973,1
4,2019-09-30 02:00:00+02:00,1908,1


In [ ]:
df.shape

(10013, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  10013 non-null  object
 1   count      10013 non-null  int64 
 2   window     10013 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 234.8+ KB


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype                                
---  ------     --------------  -----                                
 0   timestamp  10013 non-null  datetime64[ns, pytz.FixedOffset(120)]
 1   count      10013 non-null  int64                                
 2   window     10013 non-null  int64                                
dtypes: datetime64[ns, pytz.FixedOffset(120)](1), int64(2)
memory usage: 234.8 KB


In [ ]:
df=df.sort_values(by=['timestamp','window']).reset_index(drop=True)

In [ ]:
df.head()

,timestamp,count,window
0,2019-09-26 02:00:00+02:00,764,1
1,2019-09-26 02:00:00+02:00,320,2
2,2019-09-26 02:00:00+02:00,113,3
3,2019-09-26 02:00:00+02:00,101,4
4,2019-09-26 02:00:00+02:00,101,5


In [ ]:
test_start = df[df['timestamp'] == '2021-07-01 02:00:00+02:00'].index[0]
valid_start = df[df['timestamp'] == '2021-06-01 02:00:00+02:00'].index[0]

In [ ]:
train = df[:valid_start]
valid = df[valid_start:test_start]
test = df[test_start:]

In [ ]:
X_train ,y_train = train.drop('count',axis=1),train['count']
X_valid ,y_valid = valid.drop('count',axis=1),valid['count']
X_test ,y_test = test.drop('count',axis=1),test['count']

ML

In [ ]:
X_train['day'] = X_train['timestamp'].dt.day
X_train['month'] = X_train['timestamp'].dt.month
X_train['year'] = X_train['timestamp'].dt.year
X_train['dayofweek'] = X_train['timestamp'].dt.dayofweek
X_train['dayofyear'] = X_train['timestamp'].dt.dayofyear
X_train['weekofyear'] = X_train['timestamp'].dt.weekofyear
X_train['is_wknd'] = X_train['timestamp'].dt.weekday // 4
X_train['is_month_start'] = X_train['timestamp'].dt.is_month_start.astype(int)
X_train['is_month_end'] = X_train['timestamp'].dt.is_month_end.astype(int)

X_valid['day'] = X_valid['timestamp'].dt.day
X_valid['month'] = X_valid['timestamp'].dt.month
X_valid['year'] = X_valid['timestamp'].dt.year
X_valid['dayofweek'] = X_valid['timestamp'].dt.dayofweek
X_valid['dayofyear'] = X_valid['timestamp'].dt.dayofyear
X_valid['weekofyear'] = X_valid['timestamp'].dt.weekofyear
X_valid['is_wknd'] = X_valid['timestamp'].dt.weekday // 4
X_valid['is_month_start'] = X_valid['timestamp'].dt.is_month_start.astype(int)
X_valid['is_month_end'] = X_valid['timestamp'].dt.is_month_end.astype(int)

X_test['day'] = X_test['timestamp'].dt.day
X_test['month'] = X_test['timestamp'].dt.month
X_test['year'] = X_test['timestamp'].dt.year
X_test['dayofweek'] = X_test['timestamp'].dt.dayofweek
X_test['dayofyear'] = X_test['timestamp'].dt.dayofyear
X_test['weekofyear'] = X_test['timestamp'].dt.weekofyear
X_test['is_wknd'] = X_test['timestamp'].dt.weekday // 4
X_test['is_month_start'] = X_test['timestamp'].dt.is_month_start.astype(int)
X_test['is_month_end'] = X_test['timestamp'].dt.is_month_end.astype(int)

X_train.drop('timestamp',axis=1,inplace=True)
X_valid.drop('timestamp',axis=1,inplace=True)
X_test.drop('timestamp',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [ ]:
X_train.head()

,window,day,month,year,dayofweek,dayofyear,weekofyear,is_wknd,is_month_start,is_month_end
0,1,26,9,2019,3,269,39,0,0,0
1,2,26,9,2019,3,269,39,0,0,0
2,3,26,9,2019,3,269,39,0,0,0
3,4,26,9,2019,3,269,39,0,0,0
4,5,26,9,2019,3,269,39,0,0,0


In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
xgb = XGBRegressor(n_estimators=30)
score = cross_val_score(xgb,X_train,y_train,scoring='neg_mean_absolute_error',cv=5,n_jobs=-1)
-score.mean()

125.21352102171457

In [ ]:
lgb = LGBMRegressor(n_estimators=40)
score = cross_val_score(lgb,X_train,y_train,scoring='neg_mean_absolute_error',cv=5,n_jobs=-1)
-score.mean()

100.18971525853055

In [ ]:
import lightgbm as lgb
params = {'n_estimators' : 25,
        "objective" : "regression",
        "metric" : "mae"

    }
    
lgtrain = lgb.Dataset(X_train, label=y_train)
lgval = lgb.Dataset(X_valid, label=y_valid)
evals_result = {}
model = lgb.train(params, lgtrain, 
                      valid_sets=[lgtrain, lgval],
                    num_boost_round=1100,
                  early_stopping_rounds=100,
                  verbose_eval=200 ,
                      evals_result=evals_result)

Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[25]	training's l1: 80.8824	valid_1's l1: 137.691


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
ypred = model.predict(X_test,num_iteration=model.best_iteration)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_test,ypred)

177.14332889410966

In [ ]:
xgb = XGBRegressor(n_estimators=30)
xgb.fit(X_train.append(X_valid),y_train.append(y_valid))

[14:58:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=30,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
mean_absolute_error(y_test,xgb.predict(X_test))

194.1823987742906

DL

In [ ]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
m = model_lstm.fit(X_train_series, y_train, validation_data=(X_valid_series, y_valid), epochs=25)
m

Epoch 1/25
288/288 [==============================] - 3s 8ms/step - loss: 225.2943 - val_loss: 319.7914
Epoch 2/25
288/288 [==============================] - 2s 7ms/step - loss: 222.2551 - val_loss: 320.5867
Epoch 3/25
288/288 [==============================] - 2s 7ms/step - loss: 222.2233 - val_loss: 313.8754
Epoch 4/25
288/288 [==============================] - 2s 7ms/step - loss: 221.9707 - val_loss: 322.3072
Epoch 5/25
288/288 [==============================] - 2s 7ms/step - loss: 221.7323 - val_loss: 318.5791
Epoch 6/25
288/288 [==============================] - 2s 7ms/step - loss: 221.7578 - val_loss: 321.7533
Epoch 7/25
288/288 [==============================] - 2s 7ms/step - loss: 222.6334 - val_loss: 317.6229
Epoch 8/25
288/288 [==============================] - 2s 7ms/step - loss: 222.0525 - val_loss: 322.2375
Epoch 9/25
288/288 [==============================] - 2s 7ms/step - loss: 222.1264 - val_loss: 317.4513
Epoch 10/25
288/288 [==============================] - 2s 7ms/st

it seem that Lightgbm is the best

In [ ]:
test['count'] = ypred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
prediction= test.to_csv('Prediction',index=False)